In [1]:
!which python

/u/pw7nc/anaconda3/bin/python


In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

In [3]:
# Create a new model
m = gp.Model("graph2x_ilp_quad")
m.Params.LogToConsole = 0

In [4]:
# Create variables
W = m.addMVar(shape=5, vtype=GRB.BINARY, name="W")

In [5]:
W

<(5,) matrix variable>

In [6]:
m

<gurobi.Model Continuous instance graph2x_ilp_quad: 0 constrs, 0 vars, Parameter changes: LogToConsole=0>

In [7]:
sim_scores = np.array(
    [
        [1.0, 0.16, 0.2, 0.3, 0.5],
        [0.16, 1.0, 0.7, 0.4, 0.2],
        [0.2, 0.7, 1.0, 0.9, 0.25],
        [0.3, 0.4, 0.9, 1.0, 0.12],
        [0.5, 0.2, 0.25, 0.12, 1.0]
    ]
)

In [8]:
sim_scores

array([[1.  , 0.16, 0.2 , 0.3 , 0.5 ],
       [0.16, 1.  , 0.7 , 0.4 , 0.2 ],
       [0.2 , 0.7 , 1.  , 0.9 , 0.25],
       [0.3 , 0.4 , 0.9 , 1.  , 0.12],
       [0.5 , 0.2 , 0.25, 0.12, 1.  ]])

In [9]:
sim_scores_upper = np.triu(sim_scores, 1)

In [10]:
sim_scores_upper

array([[0.  , 0.16, 0.2 , 0.3 , 0.5 ],
       [0.  , 0.  , 0.7 , 0.4 , 0.2 ],
       [0.  , 0.  , 0.  , 0.9 , 0.25],
       [0.  , 0.  , 0.  , 0.  , 0.12],
       [0.  , 0.  , 0.  , 0.  , 0.  ]])

In [11]:
pred_scores = np.array(
    [0.8, 0.6, 0.5, 0.2, 0.01]
)

In [12]:
pred_scores

array([0.8 , 0.6 , 0.5 , 0.2 , 0.01])

In [13]:
alpha = 1.0

# Option 1: Use List Comprehension
# m.setObjective(
#     pred_scores @ x - sum(x[i] @ x[j] * sim_scores[i][j] for i in range(5) for j in range(i+1, 5)),
#     GRB.MAXIMIZE
# )

# Option 2: Use Matrix matmul. Note, the sim_score should only contain the upper triangle matrix
m.setObjective(
    (pred_scores @ W) - alpha * (W @ sim_scores_upper @ W),
    GRB.MAXIMIZE
)

In [14]:
np.ones(5)

array([1., 1., 1., 1., 1.])

In [15]:
N = 3
ones = np.ones(5)
m.addConstr(ones @ W == N, name="c")

<(1,) matrix constraint>

In [16]:
m

<gurobi.Model Continuous instance graph2x_ilp_quad: 0 constrs, 0 vars, Parameter changes: LogToConsole=0>

In [17]:
# Optimize model
m.optimize()

In [18]:
m

<gurobi.Model MIP instance graph2x_ilp_quad: 1 constrs, 5 vars, Parameter changes: LogToConsole=0>

In [19]:
print(W.X)
print('Obj: %g' % m.objVal)

[ 1.  1.  1. -0. -0.]
Obj: 0.84


In [20]:
m

<gurobi.Model MIP instance graph2x_ilp_quad: 1 constrs, 5 vars, Parameter changes: LogToConsole=0>

In [21]:
m.dispose()